In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, BatchNormalization, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.applications import EfficientNetB3, ResNet50

In [2]:
base_dir = '../data/telemoji'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

train_datagen = ImageDataGenerator(
      rescale=1/255,    
#     rotation_range=40,
#     width_shift_range=0.1,
#     height_shift_range=0.1,
#     zoom_range=0.2,
#     horizontal_flip=True,
#     vertical_flip=True
)

validation_datagen = ImageDataGenerator(
    rescale=1/255
)

train_generator = train_datagen.flow_from_directory(
    train_dir, 
    color_mode='grayscale',
    target_size=(48,48),
    batch_size=50,
    class_mode='categorical'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_dir,
    color_mode='grayscale',
    target_size=(48,48),
    batch_size=100,
    class_mode='categorical'
)

Found 16800 images belonging to 7 classes.
Found 7000 images belonging to 7 classes.


In [ ]:
# pretrained network
model_base = EfficientNetB3(include_top=False, 
                            weights='imagenet', input_shape=(48,48,3))
# model_base의 weight학습을 동결
model_base.trainable=False

model_base.summary()

In [ ]:
model = Sequential()

# pretrained network를 우리의 모델 앞에 추가
model.add(model_base)

model.add(Flatten())

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))

model.summary()

model.compile(optimizer=Adam(learning_rate=1e-4), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [5]:
## 다른 모델

model = Sequential()
model.add(Conv2D(64, (1, 1), padding='same', activation='relu', 
                 input_shape=(48, 48, 1)))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.25))

model.add(Conv2D(256, (5, 5),padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D((2, 2),padding="same"))
model.add(Dropout(0.25))


model.add(Flatten())

model.add(Dense(128))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(256))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(7, activation='softmax'))
model.summary()

model.compile(optimizer=Adam(learning_rate=1e-3), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 48, 48, 64)        128       
_________________________________________________________________
batch_normalization_5 (Batch (None, 48, 48, 64)        256       
_________________________________________________________________
dropout_5 (Dropout)          (None, 48, 48, 64)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 48, 48, 128)       73856     
_________________________________________________________________
batch_normalization_6 (Batch (None, 48, 48, 128)       512       
_________________________________________________________________
dropout_6 (Dropout)          (None, 48, 48, 128)       0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 48, 48, 256)      

In [7]:
history = model.fit(train_generator, steps_per_epoch=48, epochs=30,
                    validation_data=validation_generator,
                    validation_steps=20)

Epoch 1/30
48/48 [==============================] - 31s 639ms/step - loss: 1.0466 - accuracy: 0.6317 - val_loss: 1.5080 - val_accuracy: 0.4405
Epoch 2/30
48/48 [==============================] - 31s 637ms/step - loss: 1.0299 - accuracy: 0.6283 - val_loss: 1.4964 - val_accuracy: 0.4335
Epoch 3/30
48/48 [==============================] - 29s 608ms/step - loss: 0.9803 - accuracy: 0.6479 - val_loss: 1.5460 - val_accuracy: 0.4425
Epoch 4/30
48/48 [==============================] - 32s 656ms/step - loss: 0.9864 - accuracy: 0.6471 - val_loss: 1.6438 - val_accuracy: 0.4150
Epoch 5/30
48/48 [==============================] - 30s 634ms/step - loss: 0.8916 - accuracy: 0.6875 - val_loss: 1.6682 - val_accuracy: 0.4145
Epoch 6/30
48/48 [==============================] - 30s 635ms/step - loss: 0.9124 - accuracy: 0.6967 - val_loss: 1.6831 - val_accuracy: 0.4190
Epoch 7/30
48/48 [==============================] - 31s 650ms/step - loss: 0.8852 - accuracy: 0.6904 - val_loss: 1.6140 - val_accuracy: 0.4310